Cifrado Asimétrico Hash BLAKE2s(32) — ECC

In [ ]:
!pip install cryptography

In [ ]:
import os, time, base64
import pandas as pd
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

def generar_archivo(nombre, num_palabras):
    """Genera un archivo con num_palabras iguales."""
    with open(nombre, "w") as f:
        f.write(("hola " * num_palabras).strip())


def leer_archivo(nombre):
    with open(nombre, "r") as f:
        return f.read()


def ecdh_hkdf_blake2s():
    """Genera clave simétrica mediante ECDH + HKDF(BLAKE2s)"""
    private_key = ec.generate_private_key(ec.SECP256R1())
    peer_private = ec.generate_private_key(ec.SECP256R1())

    # ECDH
    shared_key = private_key.exchange(ec.ECDH(), peer_private.public_key())

    # HKDF con BLAKE2s
    derived_key = HKDF(
        algorithm=hashes.BLAKE2s(32),
        length=32,
        salt=None,
        info=b'handshake data',
    ).derive(shared_key)

    return derived_key


def cifrar(derived_key, mensaje):
    aesgcm = AESGCM(derived_key)
    nonce = os.urandom(12)
    ciphertext = aesgcm.encrypt(nonce, mensaje.encode(), None)
    return nonce, ciphertext


def descifrar(derived_key, nonce, ciphertext):
    aesgcm = AESGCM(derived_key)
    return aesgcm.decrypt(nonce, ciphertext, None).decode()

tamaños = [10, 100, 1000, 10000, 100000, 1000000, 10000000]
resultados = []

for palabras in tamaños:
    print(f"\n Procesando archivo con {palabras} palabras...")

    nombre_archivo = f"text_{palabras}.txt"

    # ----------------- ETAPA 1: Leer archivo -----------------
    t1 = time.time()
    generar_archivo(nombre_archivo, palabras)
    texto = leer_archivo(nombre_archivo)
    t2 = time.time()
    #print(texto)

    # ----------------- ETAPA 2: Generar clave ----------------
    t3 = time.time()
    derived_key = ecdh_hkdf_blake2s()
    t4 = time.time()
    #print(derived_key)

    # ----------------- ETAPA 3: Cifrar -----------------------
    t5 = time.time()
    nonce, ciphertext = cifrar(derived_key, texto)
    t6 = time.time()
    ciphertext
    #print(nonce,ciphertext)

    # ----------------- ETAPA 4: Descifrar --------------------
    t7 = time.time()
    text_plain = descifrar(derived_key, nonce, ciphertext)
    t8 = time.time()
    #print(text_plain)

    # ----------------- TABLA DE TIEMPOS ----------------------
    resultados.append([
        "ECDH+HKDF(BLAKE2s)+AES-GCM",
        palabras,
        len(texto),
        len(ciphertext),
        t2 - t1,   # Tiempo lectura
        t4 - t3,   # Tiempo clave
        t6 - t5,   # Tiempo cifrado
        t8 - t7,   # Tiempo descifrado
        t8 - t1    # Tiempo total
    ])

df = pd.DataFrame(resultados, columns=[
    "Algoritmo",
    "#Palabras",
    "#Caracteres_Entrada",
    "#Caracteres_Salida",
    "T-E1 (Leer)",
    "T-E2 (Clave)",
    "T-E3 (Cifrar)",
    "T-E4 (Descifrar)",
    "T-Total",
])

print("\n=== TABLA RESUMEN ===")
print(df)

# Guardar CSV
df.to_csv("benchmark_results.csv", index=False)
print("\nCSV generado: benchmark_results.csv")



 Procesando archivo con 10 palabras...

 Procesando archivo con 100 palabras...

 Procesando archivo con 1000 palabras...

 Procesando archivo con 10000 palabras...

 Procesando archivo con 100000 palabras...

 Procesando archivo con 1000000 palabras...

 Procesando archivo con 10000000 palabras...

=== TABLA RESUMEN ===
                    Algoritmo  #Palabras  #Caracteres_Entrada  \
0  ECDH+HKDF(BLAKE2s)+AES-GCM         10                   49   
1  ECDH+HKDF(BLAKE2s)+AES-GCM        100                  499   
2  ECDH+HKDF(BLAKE2s)+AES-GCM       1000                 4999   
3  ECDH+HKDF(BLAKE2s)+AES-GCM      10000                49999   
4  ECDH+HKDF(BLAKE2s)+AES-GCM     100000               499999   
5  ECDH+HKDF(BLAKE2s)+AES-GCM    1000000              4999999   
6  ECDH+HKDF(BLAKE2s)+AES-GCM   10000000             49999999   

   #Caracteres_Salida  T-E1 (Leer)  T-E2 (Clave)  T-E3 (Cifrar)  \
0                  65     0.005571      0.000433       0.002397   
1                 515